In [ ]:
#Here I'm going to start working on different columns to use for KMeans algorithm
from sklearn.preprocessing import Imputer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.cluster import KMeans
from sklearn import metrics
from scipy.spatial.distance import cdist
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import silhouette_samples, silhouette_score
import time

#Grab certain columns and do some pre-processing work before performing KMeans algorithm on the dataframe
qf1 = pd.get_dummies(df['teaching_hospital_id'])
qf2 = pd.get_dummies(df['amog_id'])
qf3 = df['total_amount_of_payment_usdollars']
qf3_min = qf3.min()
qf3_max = qf3.max()

qf3 -= qf3_min
qf3 /= qf3_max

frames = [qf1, qf2, qf3]
X = pd.concat(frames, axis=1)

df_info(X)

In [ ]:
#Create an "elbow" like graph based on different KMean clusters

distortions = []
K = range(2,50)

for clusters in K:
    start = time.time()
    
    k_means = KMeans(n_clusters=clusters)
    k_means.fit(X)
    
    distortions.append( sum(np.min(cdist(X, k_means.cluster_centers_, 'euclidean'), axis=1)) / X.shape[0])  
    
    end = time.time()
    print("Process took... " + str(end-start) + " seconds!")

# Plot the elbow
plt.plot(K, distortions, 'bx-')
plt.xlabel('k')
plt.ylabel('Distortion')
plt.title('The Elbow Method showing the optimal k')
plt.show()    



In [ ]:
#Calculating how much time it takes to create N clusters
start = time.time()

num_clusters = 20

k_means = KMeans(n_clusters=num_clusters)
k_means.fit(X)
end = time.time()

cluster_count = [0] * num_clusters

for row in k_means.labels_:
    cluster_count[row]+=1
    
for ind,cluster in enumerate(cluster_count):
    print(str(ind) + " " + str(cluster))
    
print("Process took... " + str(end-start) + " seconds!")

#Creating a new column in the dataframe with the results of the kmeans and importing that to a csv file.
df['kmeans'] = k_means.labels_
df.to_csv("all.csv",index=False)

In [ ]:
from itertools import combinations
import pandas as pd
import numpy as np

def apriori(trans, support=0.01, minlen=1):
    ts=pd.get_dummies(trans.unstack().dropna()).groupby(level=1).sum()

    collen, rowlen  =ts.shape

    #-------------Max leng (not used)
    #tssum=ts.sum(axis=1)
    #maxlen=int(tssum.loc[tssum.idxmax()])


    pattern = []
    for cnum in range(minlen, rowlen+1):
        for cols in combinations(ts, cnum):
            patsup = ts[list(cols)].all(axis=1).sum()
            patsup=float(patsup)/collen
            pattern.append([",".join(cols), patsup])
    sdf = pd.DataFrame(pattern, columns=["Pattern", "Support"])
    results=sdf[sdf.Support >= support]

    return results